In [1]:
%run ./importLibraries.ipynb
%run ./setup.ipynb
%run ./displayImage.ipynb
%run ./CNN_Encoder.ipynb
%run ./BioFaceUNet.ipynb
%run ./uploadImage.ipynb
%run ./scalingNet.ipynb
%run ./illuminationModel.ipynb
%run ./CameraModel.ipynb
%run ./computeLightColor.ipynb
%run ./computeSpecularities.ipynb
%run ./BiotoSpectralRef.ipynb
%run ./imageFormation.ipynb
%run ./whiteBalanced.ipynb
%run ./findT.ipynb
%run ./fromRawToRGB.ipynb
%run ./lossCalc.ipynb

cpu


In [2]:
#imgData = loadImg(40)
#still need to find actual mask
#img, gray_img = dispImg(imgData,4)
#imgDataV2 = loadImgV2("C:/Python/Python37/VMG_Candidate/img_align_celeba")

In [3]:
from torch.utils.data import DataLoader

In [4]:
#imageDataTens = torch.tensor(imgData).float().to(device)
#batch_size = 4
#X_train = imageDataTens[:32]
#X_test = imageDataTens[32:]
#loadedImageData_train = {'train': DataLoader(X_train,batch_size = batch_size, shuffle = False)}

In [5]:
torch.cuda.empty_cache()

In [6]:
#biofacenet = BioFaceNet().to(device)

In [7]:
#for X in loadedImageData_train['train']:
#    lightingParameters,b,fmel,fblood,shading,specmask = biofacenet(X)
#    weightA,weightD,CCT,Fweights,b,BGrid,fmel,fblood,Shading,specmask = ScalingNet(lightingParameters,b,fmel,fblood,shading,specmask)
#    e = illuminationModel(weightA,weightD,Fweights,CCT,illumA,illumDNorm,illumFNorm)
#    Sr,Sg,Sb = cameraModel(mu,pc,b,33)
#    lightColor = computeLightColor(e, Sr, Sg, Sb)
#    Specularities = computeSpecularities(specmask,lightColor)
#    R_total = BiotoSpectralRef(fmel,fblood,newskincolour)
#    rawAppearance,shadedDiffuse = ImageFormation(R_total, Sr,Sg,Sb,e,Specularities,Shading)
#    whiteBalanced = WhiteBalance(rawAppearance,lightColor)
#    T_RAW2XYZ = findT(Tmatrix,BGrid)
#    sRGBim = fromRawTosRGB(whiteBalanced,T_RAW2XYZ)

In [8]:
#bioFace = BioFaceUNet().to(device)
#optimizer = optim.SGD(bioFace.parameters(),lr=0.001,momentum=0.9)
#blossweight = 1e-4;  
#appweight = 1e-3; 
#Shadingweight = 1e-5; 
#sparseweight = 1e-5;

In [9]:
#for epoch in range(6):
#    running_loss = 0.0
#    for X in loadedImageData_train['train']:
#        optimizer.zero_grad()
#        lightingParameters,b,fmel,fblood,shading,specmask = bioFace(X)
#        weightA,weightD,CCT,Fweights,b,BGrid,fmel,fblood,Shading,specmask = ScalingNet(lightingParameters,b,fmel,fblood,shading,specmask)
#        e = illuminationModel(weightA,weightD,Fweights,CCT,illumA,illumDNorm,illumFNorm)
#        Sr,Sg,Sb = cameraModel(mu,pc,b,33)
#        lightColor = computeLightColor(e, Sr, Sg, Sb)
#        Specularities = computeSpecularities(specmask,lightColor)
#        R_total = BiotoSpectralRef(fmel,fblood,newskincolour)
#        rawAppearance,shadedDiffuse = ImageFormation(R_total, Sr,Sg,Sb,e,Specularities,Shading)
#        whiteBalanced = WhiteBalance(rawAppearance,lightColor)
#        T_RAW2XYZ = findT(Tmatrix,BGrid)
#        sRGBim = fromRawTosRGB(whiteBalanced,T_RAW2XYZ)
        
        #Loss
#        sRGBim_perm = sRGBim.permute(0,2,3,1)
#        bLoss = cameraLoss(b,blossweight)
#        appLoss = appearanceLoss(X,sRGBim_perm,appweight)
#        sparsLoss = sparsityLoss(Specularities,sparseweight)
#        loss = bLoss+appLoss+sparsLoss
#        running_loss = running_loss+loss.item()
#        loss.backward()
#        optimizer.step()
#        if epoch%2 == 0:
#            print("-----epoch: ",epoch)
#            print*("Loss: ", running_loss/batch_size)
        


In [10]:
#print("sRGBim shape: ",sRGBim.shape)
#print("sRGBim_perm shape: ",sRGBim_perm.shape)
#print("X shape:", X.shape)
#print("b  shape:", b.shape)

In [11]:
#bLoss1 = cameraLoss(b,blossweight)
#appLoss1 = appearanceLoss(X,sRGBim_perm,appweight)
#sparsLoss1 = sparsityLoss(Specularities,sparseweight)
#print("b loss shape:", bLoss1)
#print("app loss  shape:", appLoss1)
#print("spars Loss  shape:", sparsLoss1)

In [12]:
blossweight = 1e-4;  
appweight = 1e-3; 
Shadingweight = 1e-5; 
sparseweight = 1e-5;

In [13]:
imgDataV2 = loadImgV2("C:/Python/Python37/VMG_Candidate/img_align_celeba")

In [14]:
print(imgDataV2[0].shape)

torch.Size([3, 128, 128])


In [15]:
X_trainV2 = imgDataV2[:32]
X_testV2 = imgDataV2[32:40]
loadedImageData_trainV2 = {'train': DataLoader(X_trainV2,batch_size = 4, shuffle = False)}

In [16]:
bioFaceV2 = BioFaceUNet().to(device)
optimizerV2 = optim.SGD(bioFaceV2.parameters(),lr=0.001,momentum=0.9)

In [17]:
for epoch in range(6):
    running_loss = 0.0
    for X in loadedImageData_trainV2['train']:
        optimizerV2.zero_grad()
        lightingParameters,b,fmel,fblood,shading,specmask = bioFaceV2(X)
        weightA,weightD,CCT,Fweights,b,BGrid,fmel,fblood,Shading,specmask = ScalingNet(lightingParameters,b,fmel,fblood,shading,specmask)
        e = illuminationModel(weightA,weightD,Fweights,CCT,illumA,illumDNorm,illumFNorm)
        Sr,Sg,Sb = cameraModel(mu,pc,b,33)
        lightColor = computeLightColor(e, Sr, Sg, Sb)
        Specularities = computeSpecularities(specmask,lightColor)
        R_total = BiotoSpectralRef(fmel,fblood,newskincolour)
        rawAppearance,shadedDiffuse = ImageFormation(R_total, Sr,Sg,Sb,e,Specularities,Shading)
        whiteBalanced = WhiteBalance(rawAppearance,lightColor)
        T_RAW2XYZ = findT(Tmatrix,BGrid)
        sRGBim = fromRawTosRGB(whiteBalanced,T_RAW2XYZ)
        
        #Loss
        bLoss = cameraLoss(b,blossweight)
        appLoss = appearanceLoss(X,sRGBim,appweight)
        sparsLoss = sparsityLoss(Specularities,sparseweight)
        loss = bLoss+appLoss+sparsLoss
        running_loss = running_loss+loss.item()
        loss.backward()
        optimizerV2.step()
        if epoch%2 == 0:
            print("-----epoch: ",epoch)
            print*("Loss: ", running_loss/4)

c:\users\venen\onedrive\desktop\python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
c:\users\venen\onedrive\desktop\python39\lib\site-packages\torch\nn\functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
c:\users\venen\onedrive\desktop\python39\lib\site-packages\torch\nn\functional.py:3981: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is des

-----epoch:  0


TypeError: can't multiply sequence by non-int of type 'builtin_function_or_method'